In [20]:
import pandas as pd

# Open ratings.csv file
ratings = pd.read_csv('../ml-latest-small/ml-latest-small/ratings.csv')

In [21]:
# Open movies.csv file
movies = pd.read_csv("../ml-latest-small/ml-latest-small/movies.csv")

In [22]:
# Merge ratings and movies
user_movie_matrix = pd.merge(ratings, movies, on="movieId")
print(user_movie_matrix.head())

   userId  movieId  rating   timestamp             title  \
0       1        1     4.0   964982703  Toy Story (1995)   
1       5        1     4.0   847434962  Toy Story (1995)   
2       7        1     4.5  1106635946  Toy Story (1995)   
3      15        1     2.5  1510577970  Toy Story (1995)   
4      17        1     4.5  1305696483  Toy Story (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1  Adventure|Animation|Children|Comedy|Fantasy  
2  Adventure|Animation|Children|Comedy|Fantasy  
3  Adventure|Animation|Children|Comedy|Fantasy  
4  Adventure|Animation|Children|Comedy|Fantasy  


In [ ]:
# # Check for common movies rated by multiple users
# common_movies = ratings.groupby("title").size().reset_index(name='count')
# print(common_movies[common_movies['count'] > 1].head(10))  # Print movies rated by more than one user

In [23]:
# Map rating to scores

mapping_score = {
    0.5:-1,
    1:-1,
    1.5:-0.5,
    2:0,
    2.5:0,
    3:0,
    3.5:0.5,
    4:1,
    4.5:1.1,
    5:1.2
}

Basandoci sulle ratings presenti nel dataset vado a proiettare il grafo movie_movie che mi consente di trovare la similarità tra i film. Il vettore di mappaggio è scelto arbitrariamente e serve a decidere quanto pesano le varie ratings.

In [25]:
import networkx as nx

# Create a directed graph
user_movie_graph = nx.Graph()

# Add nodes and edges
for _, row in user_movie_matrix.iterrows():
    user_movie_graph.add_node(row["userId"], bipartite=0)
    user_movie_graph.add_node(row["title"], bipartite=1, genre=row["genres"], movieId=row["movieId"])
    # user_movie_graph.add_edge(row["userId"], row["title"], weight=row["rating"])
    user_movie_graph.add_edge(row["userId"], row["title"], weight=mapping_score[row["rating"]])

# Debug print to check the graph construction
print(f"Nodes in the graph: {list(user_movie_graph.nodes(data=True))[:10]}")
print(f"Edges in the graph: {list(user_movie_graph.edges(data=True))[:10]}")

Nodes in the graph: [(1, {'bipartite': 0}), ('Toy Story (1995)', {'bipartite': 1, 'genre': 'Adventure|Animation|Children|Comedy|Fantasy', 'movieId': 1}), (5, {'bipartite': 0}), (7, {'bipartite': 0}), (15, {'bipartite': 0}), (17, {'bipartite': 0}), (18, {'bipartite': 0}), (19, {'bipartite': 0}), (21, {'bipartite': 0}), (27, {'bipartite': 0})]
Edges in the graph: [(1, 'Toy Story (1995)', {'weight': 1}), (1, 'Grumpier Old Men (1995)', {'weight': 1}), (1, 'Heat (1995)', {'weight': 1}), (1, 'Seven (a.k.a. Se7en) (1995)', {'weight': 1.2}), (1, 'Usual Suspects, The (1995)', {'weight': 1.2}), (1, 'From Dusk Till Dawn (1996)', {'weight': 0}), (1, 'Bottle Rocket (1996)', {'weight': 1.2}), (1, 'Braveheart (1995)', {'weight': 1}), (1, 'Rob Roy (1995)', {'weight': 1.2}), (1, 'Canadian Bacon (1995)', {'weight': 1.2})]


In [26]:
users = {n for n, d in user_movie_graph.nodes(data=True) if d["bipartite"] == 0}
print(f"Users: {list(users)[:10]}")
print(f"Number of users: {len(users)}")

Users: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Number of users: 610


In [27]:
movies = {n for n, d in user_movie_graph.nodes(data=True) if d["bipartite"] == 1}
print(f"Movies: {list(movies)[:10]}")
print(f"Number of movies: {len(movies)}")

Movies: ['Zabriskie Point (1970)', 'Envy (2004)', 'House of Flying Daggers (Shi mian mai fu) (2004)', 'Super Troopers (2001)', 'Over the Top (1987)', 'Grace of My Heart (1996)', 'Pride and Glory (2008)', 'Psycho II (1983)', 'Dawn of the Planet of the Apes (2014)', 'Slasher (2004)']
Number of movies: 9719


In [28]:
print(nx.is_bipartite(user_movie_graph))
print(nx.is_connected(user_movie_graph))

True
True


In [ ]:
# user_user_graph = nx.bipartite.weighted_projected_graph(user_movie_graph, users)
# print(f"Nodes in the user-user graph: {list(user_user_graph.nodes(data=True))[:10]}")
# print(f"Edges in the user-user graph: {list(user_user_graph.edges(data=True))[:10]}")
# print(len(user_user_graph.nodes()))
# print(len(user_user_graph.edges()))

In [29]:
# Project the graph using weights
movie_movie_graph = nx.bipartite.weighted_projected_graph(user_movie_graph, movies)
print(f"Nodes in movie_movie_graph: {list(movie_movie_graph.nodes(data=True))[:10]}")
print(f"Edges in movie_movie_graph: {list(movie_movie_graph.edges(data=True))[:10]}")

Nodes in movie_movie_graph: [('Zabriskie Point (1970)', {'bipartite': 1, 'genre': 'Drama|Romance', 'movieId': 26237}), ('Envy (2004)', {'bipartite': 1, 'genre': 'Comedy', 'movieId': 7448}), ('House of Flying Daggers (Shi mian mai fu) (2004)', {'bipartite': 1, 'genre': 'Action|Drama|Romance', 'movieId': 8983}), ('Super Troopers (2001)', {'bipartite': 1, 'genre': 'Comedy|Crime|Mystery', 'movieId': 5110}), ('Over the Top (1987)', {'bipartite': 1, 'genre': 'Action|Drama', 'movieId': 7016}), ('Grace of My Heart (1996)', {'bipartite': 1, 'genre': 'Comedy|Drama', 'movieId': 988}), ('Pride and Glory (2008)', {'bipartite': 1, 'genre': 'Crime|Drama', 'movieId': 62792}), ('Psycho II (1983)', {'bipartite': 1, 'genre': 'Horror|Mystery|Thriller', 'movieId': 2902}), ('Dawn of the Planet of the Apes (2014)', {'bipartite': 1, 'genre': 'Sci-Fi', 'movieId': 112623}), ('Slasher (2004)', {'bipartite': 1, 'genre': 'Documentary', 'movieId': 27829})]
Edges in movie_movie_graph: [('Zabriskie Point (1970)', 'Fi

In [30]:
print(nx.is_connected(movie_movie_graph))

True


In [31]:
# 0: User, 1: Movie
def filter_nodes(graph: nx.Graph, node_type: int):
    return [n for n, d in graph.nodes(data=True) if d["bipartite"] == node_type]


In [32]:
def create_preference_vector(debug: bool, user_id: int, user_movie_graph: nx.Graph):
    # Get the edges for the user
    edges = {m: v for _, m, v in user_movie_graph.edges(user_id, data="weight")}
    
    if debug:
        print(f"Edges for user {user_id}: {list(edges)[:10]}")
        print(f"Number of edges for user {user_id}: {len(edges)}")
        
        for k, v in edges.items():
            print(k,v)

    # Sum of the ratings done by the user
    tot = sum(edges.values())
    
    if debug:
        print(f"Total for user {user_id}: {tot}")

    if tot > 0:
        print(f"User {user_id} has rated movies")
        return len(edges), {
            # Assign to each movie a normalized weight. The higher the rating, the higher the weight. 
            # Se provo a prendere un film che non sta in edges, mi ritorna 0 
            movie: edges.get(movie, 0) / tot
            for movie in filter_nodes(user_movie_graph, 1) # 1 : Movie
        }
    else:
        # User has not rated any movies or the sum of all weighted ratings is zero / negative
        print(f"User {user_id} has not rated any movies or the sum of all weighted ratings is zero / negative. All movies will have a weight of 1")
        return len(edges), {
            movie: 1 for movie in filter_nodes(user_movie_graph, 1) # Penso che dovremmo metterci zero come peso in questo caso
        }

Il vettore di personalizzazione assegna 0 a tutti i film non visti, mentre assegna valori normalizzati in base alla rating a tutti gli altri archi. Per come è stato costruito il vettore dei pesi delle ratings, al momento voti come 2, 2.5 e 3 vengono considerati allo stesso modo di film non visti

In [34]:
# # Test the function
# debug = True
# num, p_vec = create_preference_vector(debug, 1, user_movie_graph)

**Page Rank**

In [33]:
def predict_user(user_id, user_movie_graph: nx.Graph, movie_movie_graph: nx.Graph):
    _, p_vec = create_preference_vector(False, user_id, user_movie_graph)
    print(f"Preference vector for user {user_id}: {list(p_vec)[:10]}")  

    already_seen = [movie for movie, p in p_vec.items() if p > 0]
    # Qua uso p > 0 perchè sopra assegno 0 ad ogni film non visto e valori diversi da 0 per quelli visti
         
    print(f"Already seen movies for user {user_id}: {list(already_seen)[:10]}")  

    if len(already_seen) < 1:
        return []
    item_rank = nx.pagerank(movie_movie_graph, personalization=p_vec, alpha=0.95, weight="weight")
    print(f"Item rank for user {user_id}: {list(item_rank)[:10]}")  
    s_t = [
        x for x in sorted(
            movie_movie_graph.nodes(), key=lambda x: item_rank[x] if x in item_rank else 0, reverse=True
            )
        if x not in already_seen
        ]
    
    return s_t

**Link Prediction with Adamic Adar Index**

L'**Adamic-Adar Index** è una misura utilizzata per calcolare la _similarità_ tra due nodi in una rete (o grafo) in base ai loro _vicini comuni_. Questa misura si basa sull'idea che i vicini comuni più _rari_ (cioè quelli con un basso grado) contribuiscano _maggiormente_ alla similarità rispetto a quelli con un alto grado.  [https://neo4j.com/docs/graph-data-science/current/alpha-algorithms/adamic-adar/]

La formula per calcolare l'indice di Adamic-Adar tra due nodi $u$ e $v$ è:

$$
A(u, v) = \sum_{w \in N(u) \cap N(v)} \frac{1}{\log |N(w)|}
$$

Dove:
- $N(u)$ è l'insieme dei vicini del nodo $u$.
- $N(v)$ è l'insieme dei vicini del nodo $v$.
- $N(w)$ è l'insieme dei vicini del nodo $w$ (nodo comune tra $u$ e $v$).
- $|N(w)|$ rappresenta il grado del nodo $w$.

In altre parole, l'indice somma i contributi inversamente proporzionali alla logaritmica del grado dei vicini comuni $w$.

L'Adamic-Adar Index viene ampiamente utilizzato nella **link prediction** nei sistemi di raccomandazione, ad esempio per predire nuovi film che un utente potrebbe apprezzare. Ecco i motivi principali:

1. **Valorizzazione dei vicini rari**:
   - La presenza di vicini comuni rari è considerata più significativa nella formazione di nuovi collegamenti, poiché suggerisce una connessione più forte tra i nodi.
   - Ad esempio, in una rete sociale, avere amici comuni con pochi altri contatti può essere indicativo di una relazione più stretta.

2. **Efficienza computazionale**:
   - L'indice è relativamente semplice da calcolare per coppie di nodi e può essere implementato in modo efficace anche su grandi reti sparse.

3. **Adattabilità a diversi domini**:
   - L'indice si applica bene a reti sociali, biologiche, di co-autorship, e molte altre tipologie di grafo.

4. **Rilevanza statistica**:
   - Penalizzando i nodi con alti gradi ($\log |N(w)|$), l'Adamic-Adar Index riduce l'effetto di "hub" o nodi centrali nella rete, rendendo la misura più bilanciata.

Nella link prediction, l'indice viene utilizzato per stimare la _probabilità_ che un collegamento esista (o si formi in futuro) tra due nodi non direttamente collegati. In particolare:

1. Si calcolano gli indici Adamic-Adar per tutte le coppie di nodi non collegati.
2. Si ordinano le coppie in base ai valori di similarità.
3. Si predice che i collegamenti con i valori più alti abbiano la maggiore probabilità di esistere o di formarsi.

Immaginiamo un grafo con i seguenti nodi e connessioni:
- Nodo $u$ è connesso a $a, b$.
- Nodo $v$ è connesso a $b, c$.

Il vicinato comune tra $u$ e $v$ è costituito dal nodo $b$. Supponendo che $|N(b)| = 10$, il contributo di $b$ all'indice Adamic-Adar sarà:

$$
\frac{1}{\log 10} \approx 0.434
$$

Se ci sono più vicini comuni, i loro contributi saranno sommati per ottenere il valore finale dell'indice.

Nonostante l'efficacia in molti scenari, l'Adamic-Adar Index presenta alcune limitazioni:
- **Bias verso nodi con basso grado**: Questa caratteristica può risultare svantaggiosa in alcune reti dove gli hub giocano un ruolo cruciale.
- **Non considera i pesi degli archi**: La versione standard non tiene conto di pesi o altre proprietà degli archi, che potrebbero essere rilevanti in applicazioni specifiche.


In [37]:
#Adamic Adar iniziale che semplicemente calcola le connessioni, e l'output è 13318643

# import numpy as np
# # Calcolo dell'indice di Adamic-Adar
# def calculate_adamic_adar(graph):
#     scores = []
#     for u, v in nx.non_edges(graph):
#         common_neighbors = set(nx.common_neighbors(graph, u, v))
#         if common_neighbors:
#             score = sum(1 / np.log(len(list(graph.neighbors(w)))) for w in common_neighbors)
#             scores.append((u, v, score))
#     return scores


In [ ]:
# # Uso delle funzioni
# adamic_adar_scores = calculate_adamic_adar(user_movie_graph)

In [39]:
# print("The link predicted are # ", len(adamic_adar_scores))

The link predicted are #  13318643


In [ ]:
#NUOVA ADAMIC ADAR
import numpy as np
# Modifica per calcolare Adamic-Adar con il tipo di link
def calculate_adamic_adar(graph):
    scores = []
    for u, v in nx.non_edges(graph):
        common_neighbors = set(nx.common_neighbors(graph, u, v))
        if common_neighbors:
            score = sum(1 / np.log(len(list(graph.neighbors(w)))) for w in common_neighbors)
            # Determinare il tipo di link
            type_u = graph.nodes[u].get('bipartite', -1)
            type_v = graph.nodes[v].get('bipartite', -1)
            if type_u == 0 and type_v == 1:  # user-movie
                link_type = "user-movie"
            elif type_u == 1 and type_v == 1:  # movie-movie
                link_type = "movie-movie"
            elif type_u == 0 and type_v == 0:  # user-user (poco probabile nel tuo caso)
                link_type = "user-user"
            else:
                link_type = "unknown"
            # Aggiungere le informazioni
            scores.append((u, v, score, link_type))
    return scores

# Calcolo dei punteggi Adamic-Adar con tipo di link
adamic_adar_scores = calculate_adamic_adar(user_movie_graph)

print("The link predicted are # ", len(adamic_adar_scores))

# Salva i risultati in un CSV
adamic_adar_df = pd.DataFrame(adamic_adar_scores, columns=["Node u", "Node v", "Score", "LinkType"])
adamic_adar_df.to_csv("adamic_adar_scores.csv", index=False)

print("Adamic-Adar scores salvati in 'adamic_adar_scores.csv'")


The link predicted are #  13318643


I link predetti dalla link prediction verranno aggiunti al grafo user-movie graph [DA DECIDERE] al fine di aiutare la prediction tramite Page Ranking, arricchendola con le nuove informazioni ottenute.

In [ ]:
# Aggiunta di link predetti
def add_predicted_links(graph, predicted_edges, threshold):
    for u, v, score in predicted_edges:
        if score > threshold:
            graph.add_edge(u, v, weight=score)
    return graph


In [ ]:
user_movie_graph_extended = add_predicted_links(user_movie_graph, adamic_adar_scores, 0.5)

**Prediction**

In [ ]:
# Test the prediction
user = 10
s_t = predict_user(user, user_movie_graph, movie_movie_graph)
print(f"Predicted movies for user {user}: {s_t[:10]}")

In [ ]:
# Make all predictions
predictions = {}
for user in filter_nodes(user_movie_graph, 0):
    predictions[user] = predict_user(user, user_movie_graph, movie_movie_graph)[:10]